In [151]:
import numpy as np
import sys
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import glob
import os
import re
from collections import Counter

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# VEST VA 2020

## VEST's Documentation File

Election results from Virginia Department of Elections (https://historical.elections.virginia.gov/)  

Absentee ballots and provisional votes were reported at the county or city level throughout the state. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.  

Chesapeake City provided a breakout of nearly all votes by precinct. A small number of hand-counted ballots were not assigned to specific precincts. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.  

Precinct shapefile primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release. A significant number of precincts throughout the state have been replaced with the U.S. Census Bureau's 2020 Redistricting Data Program final release. Virginia election reports sometimes include precinct splits that are obsolete or unused for the given election cycle. These have been omitted. In cases where voters were assigned to the wrong district the de facto precinct split has been included for that election.  

The borders of Chesapeake, Norfolk, Petersburg, Prince George, Suffolk, Virginia Beach reflect de facto misalignments that were not corrected before 2020.  

The following modifications were made to recreate the 2020 precinct boundaries.  

Albemarle: Add Mechums River to match county GIS shapefile  
Alexandria City: Add Alex Renew, Olympus Condo; Adjust Lee Center/Lyles Crouch, Beatley Library/Cameron Station to match county GIS shapefile  
Arlington: Adjust Gunston/Oakridge to match county GIS shapefile  
Bristol City: Adjust Ward 2/Ward 4 to match description in municipal code  
Charles City County: Adjust District 1/District 2 boundary to match county code  
Chesapeake City: Split, merge, or adjust 26 precincts to match PDF maps of citywide realignment  
Chesterfield: Split Iron Bridge/Iron Bridge North, Skinquarter/Magnolia, Winfrees Store/Longhouse, Southside/Ridgedale, Genito/Clover Hill to match county GIS shapefile  
Covington City: Realign Ward 1, Ward 2, Ward 3 to match city PDF map and municipal code  
Culpeper: Adjust East Fairfax/Brandy Station boundary to match county GIS shapefile  
Emporia City: Adjust Precincts 1/7, Precincts 2/5 to match municipal code  
Essex: Adjust South Precinct/Central Precinct boundary to match county PDF  
Fairfax: Adjust Bull Run/Virginia Run, Shreve/Timber Lane to match county GIS shapefile  
Fredericksburg City: Adjust District 1/3 boundaries to match municipal code  
Galax City: Adjust North/South precinct boundary to match municipal GIS shapefile  
Halifax: Merge South Boston East/West; Adjust Meadville/Republican Grove to match 2011 redistricting PDF map  
Hampton City: Add US House District 2 segment of Tyler Precinct to match county PDF; Adjust 6 precincts to reflect correction of border with Newport News City  
Hanover: Adjust Blunts/Beaverdam boundary to match county PDF  
Henry: Adjust 10 precinct boundaries to align VTDs with county GIS shapefile  
Loudoun: Adjust Weller/Ashbrook, Evergreen/Sycolin Creek; Add Goshen Post, Buffalo Trail to match county GIS shapefile  
Newport News City: Adjust Sanford/Riverview boundary to match county GIS shapefile; Ajust 12 precincts to reflect correction of border with Hampton City  
Norfolk City: Merge Bramblewood into Chesterfield/Ruffner Academy; Adjust Granby/Titustown to match county PDF  
Northumberland: Merge subprecincts 3-A and 3-B to match county PDF  
Prince William: Split Piney Branch/Bristow Run, Potomac/Williams; Adjust Ben Lomond/Mullen, Freedom/Leesylvania to match county GIS shapefile  
Radford City: Split East/New River precincts to match municipal ordinance  
Rappahanock: Adjust Sperryville/Washington boundary to match county PDF  
Richmond City: Add Precinct 115, 215, 415 to match county GIS shapefile  
Richmond County: Adjust Precincts 2-1/3-1 boundary to match description in county ordinance  
Roanoke County: Adjust 12 precinct boundaries to match county GIS shapefile  
Rockingham: Split East/West Bridgewater to match municipal ordinance; Adjust East Bridgewater to match municipal boundary  
Russell: Adjust Daugherty/West Lebanon boundary to match county PDF  
Tazewell: Merge Abbs Valley/Bossevain, Adria/Gap Store, Pounding Mill/Wardell, Raven/Richlands;  Adjust nearly all precinct boundaries to align VTDs with county GIS shapefile  
Virginia Beach City: Adjust Centerville/Colonial to match county GIS shapefile  
Williamsburg City: Revise Matoaka/Stryker to match municipal PDF map and municipal code  
Wise: Adjust Big Stone Gap/East Stone Gap boundary to match county GIS shapefile  
Wythe: Adjust West Wytheville/East Wytheville boundary to match county GIS shapefile  
York: Adjust Dare/Harwoods Mill to match county GIS shapefile  

G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRERTRU - Donald J. Trump (Republican Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREOWRI - Write-in Votes  

G20USSDWAR - Mark Robert Warner (Democratic Party)  
G20USSRGAD - Daniel Mac Arthur Gade (Republican Party)  
G20USSOWRI - Write-in Votes   

## Load VEST's File

In [152]:
vest_shp = gp.read_file("./raw-from-source/VEST/va_2020/va_2020.shp")

In [153]:
vest_shp.head(10)

,COUNTYFP,LOCALITY,VTDST,PRECINCT,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,geometry
0,001,Accomack County,000101,Chincoteague,837,1618,29,2,915,1563,3,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42..."
1,001,Accomack County,000201,Atlantic,321,657,11,2,357,644,0,"POLYGON Z ((-75.59978 37.87664 0.00000, -75.59..."
2,001,Accomack County,000202,Greenbackville,516,1091,18,0,539,1054,0,"POLYGON Z ((-75.49919 37.93416 0.00000, -75.49..."
3,001,Accomack County,000301,New Church,1013,667,14,2,1003,638,2,"POLYGON Z ((-75.64987 37.92702 0.00000, -75.64..."
4,001,Accomack County,000401,Bloxom,307,462,8,0,306,447,0,"POLYGON Z ((-75.71556 37.87513 0.00000, -75.71..."
5,001,Accomack County,000402,Parksley,388,586,11,0,461,551,0,"POLYGON Z ((-75.77180 37.87512 0.00000, -75.74..."
6,001,Accomack County,000403,Saxis,124,229,0,0,155,206,2,"POLYGON Z ((-75.80121 37.91219 0.00000, -75.78..."
7,001,Accomack County,000501,Mappsville,483,225,6,2,455,227,0,"POLYGON Z ((-75.62790 37.79933 0.00000, -75.62..."
8,001,Accomack County,000502,Rue,363,125,5,0,350,130,0,"POLYGON Z ((-75.67069 37.75804 0.00000, -75.66..."
9,001,Accomack County,000602,Tangier,80,315,6,2,138,301,0,"POLYGON Z ((-76.23646 37.88660 0.00000, -76.23..."


# Election Results

## Load in Election Results

Election results from Virginia Department of Elections (https://historical.elections.virginia.gov/)

RDH Note: Data found here https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2020/

In [154]:
election_results_1= pd.read_csv("./raw-from-source/SOS/2020 March Democratic Presidential Primary.csv", index_col=False)

In [155]:
election_results_2= pd.read_csv("./raw-from-source/SOS/2020 June Republican Primary.csv", index_col=False)

In [156]:
election_results_3= pd.read_csv("./raw-from-source/SOS/2020 June Democratic Primary.csv", index_col=False)

In [157]:
election_results_1.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK COUNTY,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary


In [158]:
election_results_1.shape

(41506, 22)

In [159]:
election_results_2.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK COUNTY,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2,{72E39A01-83E7-4A1F-A743-8A0241C16F23},Member House of Representatives,{0974824B-2D74-47EB-95A9-CA5810899DFC},Primary,6/23/2020 0:00,2020 June Republican Primary


In [160]:
election_results_2.shape

(12714, 22)

In [161]:
election_results_3.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,0.0,NaN,1,{B28C7D59-123A-4A81-9926-DB24B0BBB7C3},3,ALBEMARLE COUNTY,NaN,# AB - Central Absentee Precinct,{5284A079-1078-4077-A45C-7DD847F612E4},Congressional,5.0,{72E39A01-83E7-4A1F-A743-8A0241C16F23},Member House of Representatives,{163157F7-4C6D-40E7-9BFA-B551F374B9F2},Primary,6/23/2020 0:00,2020 June Democratic Primary


In [162]:
election_results_3.shape

(3949, 22)

In [163]:
election_results = pd.concat([election_results_1, election_results_2, election_results_3], axis=0)

In [164]:
election_results.shape

(58169, 22)

In [165]:
election_results = election_results.reset_index(drop=True)

In [166]:
election_results.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK COUNTY,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2.0,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary


In [167]:
#combine candidate name columns
election_results['cand_name'] = election_results['FirstName'].astype(str) + " " + election_results['MiddleName'].astype(str) + " " + election_results['LastName'].astype(str)

In [168]:
#replace nan values
election_results['cand_name'] = election_results['cand_name'].str.replace('nan ', '')

In [169]:
#filter for presidential and senate results
election_results = election_results[election_results['OfficeTitle'].isin(['Member United States Senate']) | election_results['OfficeTitle'].isin(['President']) | election_results['OfficeTitle'].isin(['Member House of Representatives'])]

In [170]:
election_results.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName,cand_name
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK COUNTY,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2.0,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary,WRITE IN VOTES


In [171]:
election_results.shape

(58129, 23)

In [172]:
election_results.loc[(election_results['LastName'] == 'WRITE IN VOTES') & (election_results['OfficeTitle'] == 'President'), 'TOTAL_VOTES'].sum()

184.0

In [173]:
election_results['LocalityName'] = election_results['LocalityName'].str.replace(' COUNTY', '')

In [174]:
election_results.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName,cand_name
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2.0,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary,WRITE IN VOTES


In [175]:
election_results['cand_name'] = election_results['cand_name'].str.upper()

In [176]:
#Load in the nationwide FIPS file
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Virginia"]
fips_file["FIPS County"]=fips_file["FIPS County"].astype(str)

#Make the FIPS three digits
fips_file["FIPS County"]=fips_file["FIPS County"].str.zfill(3)

#Make FIPS County Name uppercase
fips_file['County Name']=fips_file['County Name'].str.upper()

#Create the dictionary
va_fips_dict = dict(zip(fips_file["County Name"],fips_file["FIPS County"]))
#Create list of fips codes
fips_file["unique_ID"] =  "51" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()

In [177]:
#Add FIPS columns
election_results["COUNTYFP"] = election_results["LocalityName"].map(va_fips_dict).fillna(election_results["LocalityName"])
election_results['DistrictName'] = election_results['DistrictName'].astype(int).astype(str)

In [178]:
election_results.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName,cand_name,COUNTYFP
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary,WRITE IN VOTES,001


In [179]:
election_results['join_col'] = election_results['COUNTYFP'] + election_results['PrecinctName']
election_results['cand_name_title'] = election_results['cand_name'] + " for " + election_results['OfficeTitle']
election_results['cand_name_title'] = np.where(election_results['OfficeTitle'] == 'Member House of Representatives', election_results['cand_name'] + " for " + election_results['OfficeTitle'] + election_results['DistrictName'], election_results['cand_name'] + " for " + election_results['OfficeTitle'])

In [180]:
election_results.head(1)

,CandidateUid,FirstName,MiddleName,LastName,Suffix,TOTAL_VOTES,Party,WriteInVote,LocalityUid,LocalityCode,LocalityName,PrecinctUid,PrecinctName,DistrictUid,DistrictType,DistrictName,OfficeUid,OfficeTitle,ElectionUid,ElectionType,ElectionDate,ElectionName,cand_name,COUNTYFP,join_col,cand_name_title
0,NaN,NaN,NaN,WRITE IN VOTES,NaN,NaN,NaN,1,{15B7E141-2D1D-44C2-A50A-AAE021BC9B7D},1,ACCOMACK,NaN,# AB - Central Absentee Precinct,{8967B9DD-E94E-4964-A919-30E520165984},Congressional,2,{67312DDD-CED6-4AB3-B269-E60728C664F6},President,{24C5731F-190B-477F-8A12-847DB54DCBB5},Primary,3/3/2020 0:00,2020 March Democratic Presidential Primary,WRITE IN VOTES,001,001# AB - Central Absentee Precinct,WRITE IN VOTES for President


In [181]:
election_results.cand_name_title.unique()

array(['WRITE IN VOTES for President', 'AMY J. KLOBUCHAR for President',
       'ANDREW YANG for President', 'BERNARD SANDERS for President',
       'CORY A BOOKER for President', 'DEVAL PATRICK for President',
       'ELIZABETH WARREN for President',
       'JOSEPH ROBINETTE BIDEN for President',
       'JULI�N CASTRO for President',
       'MARIANNE D WILLIAMSON for President',
       'MICHAEL F. BENNET for President',
       'MICHAEL R BLOOMBERG for President',
       'PETER P. M. BUTTIGIEG for President',
       'THOMAS STEYER for President', 'TULSI GABBARD for President',
       'WRITE IN VOTES for Member House of Representatives2',
       'WRITE IN VOTES for Member United States Senate',
       'ALISSA ANNE BALDWIN for Member United States Senate',
       'BENITO LOYOLA for Member House of Representatives2',
       'DANIEL MAC ARTHUR GADE for Member United States Senate',
       'JAROME BELL for Member House of Representatives2',
       'SCOTT WILLIAM TAYLOR for Member House of R

In [182]:
pivot_20 = pd.pivot_table(election_results, index = ['PrecinctName','LocalityName','COUNTYFP'], columns = ['cand_name_title'], values ='TOTAL_VOTES', aggfunc='sum')
pivot_20 = pivot_20.rename(columns={'WRITE IN VOTES for President':'P20PREOWRI', 
                                    'AMY J. KLOBUCHAR for President':'P20PREDKLO',
                                    'ANDREW YANG for President':'P20PREDYAN', 
                                    'BERNARD SANDERS for President':'P20PREDSAN',
                                    'CORY A BOOKER for President':'P20PREDBOO', 
                                    'DEVAL PATRICK for President':'P20PREDPAT',
                                    'ELIZABETH WARREN for President':'P20PREDWAR',
                                    'JOSEPH ROBINETTE BIDEN for President':'P20PREDBID',
                                    'JULI�N CASTRO for President':'P20PREDCAS',
                                    'MARIANNE D WILLIAMSON for President':'P20PREDWIL',
                                    'MICHAEL F. BENNET for President':'P20PREDBEN',
                                    'MICHAEL R BLOOMBERG for President':'P20PREDBLO',
                                    'PETER P. M. BUTTIGIEG for President':'P20PREDBUT',
                                    'THOMAS STEYER for President':'P20PREDSTE', 
                                    'TULSI GABBARD for President':'P20PREDGAB',
                                    'WRITE IN VOTES for Member House of Representatives2':'PCON02OWRI',
                                    'WRITE IN VOTES for Member United States Senate':'P20USSOWRI',
                                    'ALISSA ANNE BALDWIN for Member United States Senate':'P20USSRBAL',
                                    'BENITO LOYOLA for Member House of Representatives2':'PCON02RLOY',
                                    'DANIEL MAC ARTHUR GADE for Member United States Senate':'P20USSRGAD',
                                    'JAROME BELL for Member House of Representatives2':'PCON02RBEL',
                                    'SCOTT WILLIAM TAYLOR for Member House of Representatives2':'PCON02RTAY',
                                    'THOMAS A SPECIALE for Member United States Senate':'P20USSRSPE',
                                    'WRITE IN VOTES for Member House of Representatives3':'PCON03OWRI',
                                    'GEORGE MICHAEL YACUS for Member House of Representatives3':'PCON03RYAC',
                                    'JOHN HARRISON MADISON DOWNS for Member House of Representatives3':'PCON03RDOW',
                                    'JOHN WILLIAM COLLICK for Member House of Representatives3':'PCON03RCOL',
                                    'WRITE IN VOTES for Member House of Representatives5':'PCON05OWRI',
                                    'BRYANT CAMERON WEBB for Member House of Representatives5':'PCON05DWEB',
                                    'CLAIRE CARMEN RUSSO for Member House of Representatives5':'PCON05DRUS',
                                    'JOHN DANIEL LESINSKI for Member House of Representatives5':'PCON05DLES',
                                    'ROGER DEAN HUFFSTETLER for Member House of Representatives5':'PCON05DHUF',
                                    'WRITE IN VOTES for Member House of Representatives1':'PCON01OWRI',
                                    'LAVANGELENE AEREKA WILLIAMS for Member House of Representatives1':'PCON01DWIL',
                                    'QASIM RASHID for Member House of Representatives1':'PCON01DRAS',
                                    'WRITE IN VOTES for Member House of Representatives4':'PCON04OWRI',
                                    'ASTON DONALD MCEACHIN for Member House of Representatives4':'PCON04DMCE',
                                    'RONGERLIS CAZEL LEVINE for Member House of Representatives4':'PCON04DLEV',
                                    'WRITE IN VOTES for Member House of Representatives11':'PCON11OWRI',
                                    'GERALD EDWARD CONNOLLY for Member House of Representatives11':'PCON11DCON',
                                    'ZAINAB MASOOMA MOHSINI for Member House of Representatives11':'PCON11DMOH'})

pivot_20 = pivot_20[['P20PREOWRI', 'P20PREDKLO','P20PREDYAN', 'P20PREDSAN','P20PREDBOO', 
                                    'P20PREDPAT','P20PREDWAR','P20PREDBID','P20PREDCAS',
                                    'P20PREDWIL','P20PREDBEN','P20PREDBLO','P20PREDBUT',
                                    'P20PREDSTE','P20PREDGAB','PCON02OWRI','P20USSOWRI',
                                    'P20USSRBAL','PCON02RLOY','P20USSRGAD','PCON02RBEL',
                                    'PCON02RTAY','P20USSRSPE','PCON03OWRI','PCON03RYAC',
                                    'PCON03RDOW','PCON03RCOL','PCON05OWRI','PCON05DWEB',
                                    'PCON05DRUS','PCON05DLES','PCON05DHUF','PCON01OWRI',
                                    'PCON01DWIL','PCON01DRAS','PCON04OWRI','PCON04DMCE',
                                    'PCON04DLEV','PCON11OWRI','PCON11DCON','PCON11DMOH']]
pivot_20.reset_index(inplace=True)
pivot_20 = pivot_20.fillna(0)
pivot_20.head(10)

cand_name_title,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH
0,# AB - Central Absentee Precinct,ACCOMACK,001,0.0,9.0,0.0,25.0,0.0,0.0,25.0,42.0,0.0,0.0,0.0,32.0,26.0,1.0,3.0,0.0,0.0,39.0,47.0,62.0,19.0,57.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,# AB - Central Absentee Precinct,ALBEMARLE,003,0.0,163.0,7.0,294.0,2.0,0.0,270.0,246.0,0.0,0.0,2.0,227.0,185.0,6.0,5.0,0.0,0.0,67.0,0.0,277.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,0.0,4149.0,1152.0,364.0,968.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,# AB - Central Absentee Precinct,ALEXANDRIA CITY,510,1.0,257.0,11.0,468.0,5.0,1.0,675.0,667.0,2.0,1.0,1.0,540.0,519.0,12.0,11.0,0.0,2.0,225.0,0.0,657.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,# AB - Central Absentee Precinct,ALLEGHANY,005,0.0,4.0,0.0,4.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,7.0,8.0,1.0,0.0,0.0,0.0,8.0,0.0,22.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,# AB - Central Absentee Precinct,AMELIA,007,0.0,3.0,2.0,9.0,0.0,1.0,3.0,14.0,0.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,0.0,11.0,0.0,51.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,# AB - Central Absentee Precinct,AMHERST,009,0.0,7.0,2.0,14.0,1.0,0.0,8.0,25.0,0.0,0.0,1.0,6.0,8.0,1.0,1.0,0.0,0.0,20.0,0.0,66.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,# AB - Central Absentee Precinct,APPOMATTOX,011,0.0,2.0,0.0,9.0,1.0,1.0,3.0,11.0,0.0,1.0,0.0,3.0,7.0,0.0,0.0,0.0,0.0,6.0,0.0,40.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,46.0,15.0,11.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,# AB - Central Absentee Precinct,ARLINGTON,013,0.0,491.0,24.0,915.0,2.0,3.0,1435.0,1065.0,1.0,2.0,2.0,815.0,937.0,15.0,17.0,0.0,1.0,148.0,0.0,551.0,0.0,0.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,# AB - Central Absentee Precinct,AUGUSTA,015,0.0,25.0,1.0,37.0,0.0,0.0,29.0,46.0,0.0,2.0,0.0,19.0,31.0,2.0,2.0,0.0,0.0,48.0,0.0,213.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,# AB - Central Absentee Precinct,BATH,017,0.0,3.0,0.0,2.0,0.0,0.0,8.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,5.0,0.0,14.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
election_results = pivot_20

In [184]:
election_results.sum(axis=0)

cand_name_title
PrecinctName    # AB - Central Absentee Precinct# AB - Central...
LocalityName    ACCOMACKALBEMARLEALEXANDRIA CITYALLEGHANYAMELI...
COUNTYFP        0010035100050070090110130150170190210235200250...
P20PREOWRI                                                  184.0
P20PREDKLO                                                 8414.0
P20PREDYAN                                                 3361.0
P20PREDSAN                                               306388.0
P20PREDBOO                                                 1910.0
P20PREDPAT                                                  370.0
P20PREDWAR                                               142546.0
P20PREDBID                                               705501.0
P20PREDCAS                                                  691.0
P20PREDWIL                                                  902.0
P20PREDBEN                                                 1437.0
P20PREDBLO                                               128

In [185]:
races = [x for x in election_results.columns if 'P20' in x]
races

['P20PREOWRI',
 'P20PREDKLO',
 'P20PREDYAN',
 'P20PREDSAN',
 'P20PREDBOO',
 'P20PREDPAT',
 'P20PREDWAR',
 'P20PREDBID',
 'P20PREDCAS',
 'P20PREDWIL',
 'P20PREDBEN',
 'P20PREDBLO',
 'P20PREDBUT',
 'P20PREDSTE',
 'P20PREDGAB',
 'P20USSOWRI',
 'P20USSRBAL',
 'P20USSRGAD',
 'P20USSRSPE']

In [186]:
con_races = [x for x in election_results.columns if 'PCON' in x]
con_races

['PCON02OWRI',
 'PCON02RLOY',
 'PCON02RBEL',
 'PCON02RTAY',
 'PCON03OWRI',
 'PCON03RYAC',
 'PCON03RDOW',
 'PCON03RCOL',
 'PCON05OWRI',
 'PCON05DWEB',
 'PCON05DRUS',
 'PCON05DLES',
 'PCON05DHUF',
 'PCON01OWRI',
 'PCON01DWIL',
 'PCON01DRAS',
 'PCON04OWRI',
 'PCON04DMCE',
 'PCON04DLEV',
 'PCON11OWRI',
 'PCON11DCON',
 'PCON11DMOH']

In [187]:
all_races = [x for x in election_results.columns if 'P' in x]
all_races = all_races[2:]
all_races

['P20PREOWRI',
 'P20PREDKLO',
 'P20PREDYAN',
 'P20PREDSAN',
 'P20PREDBOO',
 'P20PREDPAT',
 'P20PREDWAR',
 'P20PREDBID',
 'P20PREDCAS',
 'P20PREDWIL',
 'P20PREDBEN',
 'P20PREDBLO',
 'P20PREDBUT',
 'P20PREDSTE',
 'P20PREDGAB',
 'PCON02OWRI',
 'P20USSOWRI',
 'P20USSRBAL',
 'PCON02RLOY',
 'P20USSRGAD',
 'PCON02RBEL',
 'PCON02RTAY',
 'P20USSRSPE',
 'PCON03OWRI',
 'PCON03RYAC',
 'PCON03RDOW',
 'PCON03RCOL',
 'PCON05OWRI',
 'PCON05DWEB',
 'PCON05DRUS',
 'PCON05DLES',
 'PCON05DHUF',
 'PCON01OWRI',
 'PCON01DWIL',
 'PCON01DRAS',
 'PCON04OWRI',
 'PCON04DMCE',
 'PCON04DLEV',
 'PCON11OWRI',
 'PCON11DCON',
 'PCON11DMOH']

## Allocate Absentee Votes

In [188]:
def allocate_absentee(df_receiving_votes,df_allocating,column_list,col_allocating):
    original_cols = list(df_receiving_votes.columns)
    
    #Add in the "Total Votes column"
    df_receiving_votes.loc[:,"Total_Votes"]=0
    for race in column_list:
        df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
    
    #Create the needed dataframes
    precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
    precinct_specific_totals.reset_index(drop=False,inplace=True)
    to_dole_out_totals = pd.DataFrame(df_allocating.groupby([col_allocating]).sum())
    to_dole_out_totals.reset_index(drop=False,inplace=True)
    
    #Print out any instances where the allocation, as written, won't work
    special_allocation_needed = []
    for index, row in precinct_specific_totals.iterrows():
        for race in column_list:
            if (row[race]==0):
                race_district = row[col_allocating]
                if race_district in to_dole_out_totals[col_allocating].unique():
                    to_allocate = int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==race_district][race])
                    if (to_allocate != 0):
                        special_allocation_needed.append([race_district,race])
    
    #Create some new columns for each of these races to deal with the allocation
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        df_receiving_votes.loc[:,add_var]=0.0
        df_receiving_votes.loc[:,rem_var]=0.0
        df_receiving_votes.loc[:,floor_var]=0.0

    #Iterate over the rows
    #Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
    for index, row in df_receiving_votes.iterrows():
        if row[col_allocating] in to_dole_out_totals[col_allocating].unique():
            for race in column_list:
                add_var = race+"_add"
                rem_var = race+"_rem"
                floor_var = race+"_floor"
                #Grab the district
                county_id = row[col_allocating]
                if [county_id,race] in special_allocation_needed:
                    #Get the denominator for the allocation - the summed "total votes" for precincts in that grouping
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id]["Total_Votes"]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the "total votes" for this particular precinct
                    val = df_receiving_votes.at[index,"Total_Votes"]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                else:
                    #Get the denominator for the allocation (the precinct vote totals)
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id][race]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the vote totals for this race in this precinct
                    val = df_receiving_votes.at[index,race]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                if ((float(denom)==0)):
                    vote_share = 0
                else:
                    vote_share = (float(val)/float(denom))*float(numer)
                df_receiving_votes.at[index,add_var] = vote_share
                #Take the decimal remainder of the allocation
                df_receiving_votes.at[index,rem_var] = vote_share%1
                #Take the floor of the allocation
                df_receiving_votes.at[index,floor_var] = np.floor(vote_share)

    #After the first pass through, get the sums of the races by district to assist in the rounding            
    first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())

    #Now we want to iterate district by district to work on rounding
    county_list = list(to_dole_out_totals[col_allocating].unique()) 

    #Iterate over the district
    for county in county_list:
        for race in column_list:
            add_var = race+"_add"
            rem_var = race+"_rem"
            floor_var = race+"_floor"
            #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
            to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
            #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
            for index in df_receiving_votes.loc[df_receiving_votes[col_allocating]==county][rem_var].nlargest(to_go).index:
                df_receiving_votes.at[index,add_var] = np.ceil(df_receiving_votes.at[index,add_var])

    #Iterate over every race again
    for race in column_list:
        add_var = race+"_add"
        #Round every allocation down to not add fractional votes
        df_receiving_votes.loc[:,add_var]=np.floor(df_receiving_votes.loc[:,add_var])
        df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
        
    df_receiving_votes = df_receiving_votes[original_cols]
    
    return df_receiving_votes

In [189]:
searchfor = ['Absentee', 'Provisional']
in_sos =  election_results[election_results["PrecinctName"].str.contains('|'.join(searchfor))]
in_sos = in_sos.groupby(by=["COUNTYFP"]).sum().reset_index()
in_sos

cand_name_title,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH
0,001,0.0,9.0,0.0,25.0,0.0,0.0,25.0,42.0,0.0,0.0,0.0,32.0,26.0,1.0,3.0,0.0,0.0,40.0,48.0,63.0,19.0,60.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,003,0.0,163.0,7.0,300.0,2.0,0.0,273.0,250.0,0.0,0.0,2.0,227.0,185.0,6.0,5.0,0.0,0.0,69.0,0.0,285.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,1.0,4219.0,1165.0,373.0,979.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,005,0.0,4.0,0.0,4.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,7.0,8.0,1.0,0.0,0.0,0.0,8.0,0.0,22.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,007,0.0,3.0,2.0,9.0,0.0,1.0,3.0,14.0,0.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,0.0,11.0,0.0,54.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,009,0.0,7.0,2.0,14.0,1.0,0.0,8.0,25.0,0.0,0.0,1.0,6.0,8.0,1.0,1.0,0.0,0.0,25.0,0.0,68.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,011,0.0,2.0,0.0,9.0,1.0,1.0,3.0,11.0,0.0,1.0,0.0,3.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,40.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,47.0,15.0,11.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,013,0.0,491.0,24.0,936.0,2.0,3.0,1449.0,1093.0,1.0,2.0,2.0,822.0,937.0,15.0,17.0,0.0,1.0,152.0,0.0,558.0,0.0,0.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,015,0.0,25.0,1.0,37.0,0.0,0.0,30.0,47.0,0.0,2.0,0.0,19.0,31.0,2.0,2.0,0.0,0.0,51.0,0.0,227.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,017,0.0,3.0,0.0,2.0,0.0,0.0,8.0,5.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,5.0,0.0,16.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,019,0.0,14.0,2.0,43.0,0.0,0.0,24.0,42.0,0.0,0.0,0.0,26.0,24.0,0.0,3.0,0.0,0.0,50.0,0.0,275.0,0.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,144.0,90.0,29.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
election_results = election_results[~election_results["PrecinctName"].str.contains('|'.join(searchfor))]
election_results = allocate_absentee(election_results,in_sos,all_races,'COUNTYFP')
print('Done')

C:\Users\davey\anaconda3\envs\geo\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\davey\anaconda3\envs\geo\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\davey\anaconda3\envs\geo\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Done


In [191]:
election_results.sum(axis=0)

cand_name_title
PrecinctName    001 - ARLINGTON001 - CENTRAL001 - CHESAPEAKE00...
LocalityName    ARLINGTONPOQUOSON CITYMATHEWSMANASSAS CITYGALA...
COUNTYFP        0137351156836407507207405705205306105506780918...
P20PREOWRI                                                  184.0
P20PREDKLO                                                 8414.0
P20PREDYAN                                                 3361.0
P20PREDSAN                                               306388.0
P20PREDBOO                                                 1910.0
P20PREDPAT                                                  370.0
P20PREDWAR                                               142546.0
P20PREDBID                                               705501.0
P20PREDCAS                                                  691.0
P20PREDWIL                                                  902.0
P20PREDBEN                                                 1437.0
P20PREDBLO                                               128

In [192]:
election_results['VTDST'] = election_results['PrecinctName'].str.split(' - ').str[0]
election_results['VTDST'] = election_results['VTDST'].str.zfill(6)
election_results['Precinct'] = election_results['PrecinctName'].str.split(' - ').str[1]

In [193]:
election_results.head(1)

cand_name_title,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct
261,001 - ARLINGTON,ARLINGTON,013,0.0,5.0,4.0,331.0,3.0,2.0,142.0,421.0,4.0,0.0,5.0,87.0,26.0,0.0,7.0,0.0,0.0,32.0,0.0,42.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000001,ARLINGTON


In [194]:
vest_shp.head(1)

,COUNTYFP,LOCALITY,VTDST,PRECINCT,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,geometry
0,001,Accomack County,000101,Chincoteague,837,1618,29,2,915,1563,3,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42..."


In [195]:
vest_shp['PRECINCT'] = vest_shp['PRECINCT'].str.upper()

In [196]:
election_results['join_col'] = election_results['COUNTYFP'] + '_' + election_results['VTDST']
vest_shp['join_col'] = vest_shp['COUNTYFP'] + '_' + vest_shp['VTDST']
election_results.head(1)

cand_name_title,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,join_col
261,001 - ARLINGTON,ARLINGTON,013,0.0,5.0,4.0,331.0,3.0,2.0,142.0,421.0,4.0,0.0,5.0,87.0,26.0,0.0,7.0,0.0,0.0,32.0,0.0,42.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000001,ARLINGTON,013_000001


In [197]:
ids = vest_shp["join_col"]
vest_shp[ids.isin(ids[ids.duplicated()])]

,COUNTYFP,LOCALITY,VTDST,PRECINCT,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,geometry,join_col
2428,059,Fairfax County,000513,SAINT ALBANS (CD 8),515,184,12,5,502,209,0,"POLYGON Z ((-77.17880 38.82896 0.00000, -77.17...",059_000513
2429,059,Fairfax County,000513,SAINT ALBANS (CD 11),1076,354,15,9,1100,348,0,"MULTIPOLYGON Z (((-77.17507 38.82548 0.00000, ...",059_000513
2430,650,Hampton City,000215,TYLER (CD 3),724,127,12,0,723,131,0,"POLYGON Z ((-76.35832 37.05238 0.00000, -76.35...",650_000215
2431,650,Hampton City,000215,TYLER (CD 2),24,11,0,0,23,12,0,"POLYGON Z ((-76.36283 37.05127 0.00000, -76.36...",650_000215
2432,089,Henry County,000505,DYERS STORE (CD 9),130,388,8,2,142,375,0,"POLYGON Z ((-79.76490 36.73391 0.00000, -79.76...",089_000505
2433,089,Henry County,000505,DYERS STORE (CD 5),38,58,0,0,43,53,0,"POLYGON Z ((-79.74112 36.76931 0.00000, -79.74...",089_000505
2434,089,Henry County,000304,MOUNT OLIVET (CD 5),565,565,7,0,588,523,0,"POLYGON Z ((-79.84036 36.68921 0.00000, -79.84...",089_000304
2435,089,Henry County,000304,MOUNT OLIVET (CD 9),3,0,0,0,3,0,0,"MULTIPOLYGON Z (((-79.84125 36.68480 0.00000, ...",089_000304
2436,095,James City County,000504,ROBERTS D (CD 2),1232,1317,54,18,1287,1337,2,"POLYGON Z ((-76.74698 37.23822 0.00000, -76.74...",095_000504
2437,095,James City County,000504,ROBERTS D (CD 1),0,6,0,0,0,6,0,"POLYGON Z ((-76.74957 37.23549 0.00000, -76.74...",095_000504


In [198]:
vest_shp_2 = vest_shp.dissolve(by='join_col', aggfunc='sum')

In [199]:
vest_shp_2.reset_index(inplace=True)

In [200]:
vest_shp_2.head(1)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837,1618,29,2,915,1563,3


## Join Election Results to VEST

In [201]:
join_attempt_one = pd.merge(vest_shp_2,election_results,how="outer",on="join_col",indicator=True)
join_attempt_one["_merge"].value_counts()

both          2395
left_only       59
right_only      58
Name: _merge, dtype: int64

In [202]:
join_attempt_one[join_attempt_one['_merge'] != 'both'][['Precinct','join_col']].to_csv('./test123.csv')

In [203]:
join_attempt_one.head(1)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837.0,1618.0,29.0,2.0,915.0,1563.0,3.0,101 - CHINCOTEAGUE,ACCOMACK,001,0.0,2.0,1.0,85.0,0.0,0.0,35.0,209.0,0.0,1.0,0.0,39.0,3.0,0.0,1.0,0.0,0.0,36.0,88.0,144.0,28.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,CHINCOTEAGUE,both


## Adjust join_col names

In [204]:
source_vest_names_dict = {'003_000499': '003_000407',
 '003_000599': '003_000507',
 '005_005101': '005_000101',
 '005_005103': '005_000103',
 '005_005201': '005_000201',
 '005_005202': '005_000202',
 '005_005301': '005_000301',
 '005_005302': '005_000302',
 '005_005401': '005_000401',
 '005_005402': '005_000402',
 '005_005501': '005_000501',
 '005_005502': '005_000502',
 '005_005601': '005_000601',
 '005_005701': '005_000701',
 '017_017101': '017_000101',
 '017_017102': '017_000102',
 '017_017201': '017_000201',
 '017_017301': '017_000301',
 '017_017302': '017_000302',
 '017_017401': '017_000401',
 '017_017501': '017_000501',
 '027_027102': '027_000102',
 '027_027103': '027_000103',
 '027_027201': '027_000201',
 '027_027202': '027_000202',
 '027_027203': '027_000203',
 '027_027301': '027_000301',
 '027_027302': '027_000302',
 '027_027403': '027_000403',
 '027_027404': '027_000404',
 '027_027501': '027_000501',
 '027_027502': '027_000502',
 '027_027503': '027_000503',
 '027_027601': '027_000601',
 '027_027602': '027_000602',
 '027_027603': '027_000603',
 '027_027701': '027_000701',
 '027_027702': '027_000702',
 '029_029101': '029_000101',
 '029_029102': '029_00102 ',
 '029_029201': '029_000201',
 '029_029301': '029_000301',
 '029_029302': '029_000302',
 '029_029401': '029_000401',
 '029_029501': '029_000501',
 '029_029502': '029_000502',
 '029_029601': '029_000601',
 '029_029701': '029_000701',
 '135_135101': '135_000101',
 '135_135102': '135_000102',
 '135_135201': '135_000201',
 '135_135202': '135_000202',
 '135_135301': '135_000301',
 '135_135302': '135_000302',
 '135_135401': '135_000401',
 '135_135402': '135_000402',
 '135_135501': '135_000501',
 '640_000002': '640_000005'}
vest_shp_2["join_col"]=vest_shp_2["join_col"].map(source_vest_names_dict).fillna(vest_shp_2["join_col"])

## Join attempt two

In [205]:
join_attempt_two = pd.merge(vest_shp_2,election_results,how="outer",on="join_col",indicator=True)
print(join_attempt_two["_merge"].value_counts())

both          2453
left_only        1
right_only       0
Name: _merge, dtype: int64


In [206]:
join_attempt_two[join_attempt_two['_merge'] != 'both'][['Precinct','join_col']]

,Precinct,join_col
615,NaN,059_000700


In [207]:
# Fairfax Court does not have any election results

In [208]:
join_attempt_two.head(5)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837,1618,29,2,915,1563,3,101 - CHINCOTEAGUE,ACCOMACK,001,0.0,2.0,1.0,85.0,0.0,0.0,35.0,209.0,0.0,1.0,0.0,39.0,3.0,0.0,1.0,0.0,0.0,36.0,88.0,144.0,28.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,CHINCOTEAGUE,both
1,001_000201,"POLYGON Z ((-75.59978 37.87664 0.00000, -75.59...",321,657,11,2,357,644,0,201 - ATLANTIC,ACCOMACK,001,0.0,2.0,0.0,21.0,0.0,0.0,8.0,64.0,0.0,2.0,0.0,13.0,0.0,0.0,5.0,0.0,0.0,27.0,32.0,85.0,30.0,77.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000201,ATLANTIC,both
2,001_000202,"POLYGON Z ((-75.49919 37.93416 0.00000, -75.49...",516,1091,18,0,539,1054,0,202 - GREENBACKVILLE,ACCOMACK,001,0.0,2.0,1.0,57.0,1.0,0.0,21.0,123.0,0.0,0.0,0.0,19.0,10.0,0.0,13.0,0.0,0.0,33.0,62.0,116.0,40.0,84.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000202,GREENBACKVILLE,both
3,001_000301,"POLYGON Z ((-75.64987 37.92702 0.00000, -75.64...",1013,667,14,2,1003,638,2,301 - NEW CHURCH,ACCOMACK,001,0.0,0.0,0.0,26.0,2.0,0.0,7.0,186.0,1.0,0.0,0.0,43.0,0.0,0.0,1.0,0.0,0.0,24.0,29.0,81.0,44.0,70.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000301,NEW CHURCH,both
4,001_000401,"POLYGON Z ((-75.71556 37.87513 0.00000, -75.71...",307,462,8,0,306,447,0,401 - BLOXOM,ACCOMACK,001,0.0,2.0,0.0,20.0,0.0,0.0,9.0,37.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,19.0,21.0,64.0,28.0,52.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000401,BLOXOM,both


In [209]:
final_join = join_attempt_two

In [210]:
join_attempt_two[join_attempt_two['join_col']=='059_000700']

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge
615,059_000700,"POLYGON Z ((-77.31648 38.84615 0.00000, -77.31...",0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [211]:
join_attempt_two.at[615,'Precinct']='Fairfax Court'
join_attempt_two.at[615,'COUNTYFP']='059'
join_attempt_two.at[615,'VTDST']='000700'
join_attempt_two.at[615,'P20PREOWRI']=0
join_attempt_two.at[615,'P20PREDKLO']=0
join_attempt_two.at[615,'P20PREDYAN']=0
join_attempt_two.at[615,'P20PREDSAN']=0
join_attempt_two.at[615,'P20PREDBOO']=0
join_attempt_two.at[615,'P20PREDPAT']=0
join_attempt_two.at[615,'P20PREDWAR']=0
join_attempt_two.at[615,'P20PREDBID']=0
join_attempt_two.at[615,'P20PREDCAS']=0
join_attempt_two.at[615,'P20PREDWIL']=0
join_attempt_two.at[615,'P20PREDBEN']=0
join_attempt_two.at[615,'P20PREDBLO']=0
join_attempt_two.at[615,'P20PREDBUT']=0
join_attempt_two.at[615,'P20PREDSTE']=0
join_attempt_two.at[615,'P20PREDGAB']=0
join_attempt_two.at[615,'PCON02OWRI']=0
join_attempt_two.at[615,'P20USSOWRI']=0
join_attempt_two.at[615,'P20USSRBAL']=0
join_attempt_two.at[615,'PCON02RLOY']=0
join_attempt_two.at[615,'P20USSRGAD']=0
join_attempt_two.at[615,'PCON02RBEL']=0
join_attempt_two.at[615,'PCON02RTAY']=0
join_attempt_two.at[615,'P20USSRSPE']=0
join_attempt_two.at[615,'PCON03OWRI']=0
join_attempt_two.at[615,'PCON03RYAC']=0
join_attempt_two.at[615,'PCON03RDOW']=0
join_attempt_two.at[615,'PCON03RCOL']=0
join_attempt_two.at[615,'PCON05OWRI']=0
join_attempt_two.at[615,'PCON05DWEB']=0
join_attempt_two.at[615,'PCON05DRUS']=0
join_attempt_two.at[615,'PCON05DLES']=0
join_attempt_two.at[615,'PCON05DHUF']=0
join_attempt_two.at[615,'PCON01OWRI']=0
join_attempt_two.at[615,'PCON01DWIL']=0
join_attempt_two.at[615,'PCON01DRAS']=0
join_attempt_two.at[615,'PCON04OWRI']=0
join_attempt_two.at[615,'PCON04DMCE']=0
join_attempt_two.at[615,'PCON04DLEV']=0
join_attempt_two.at[615,'PCON11OWRI']=0
join_attempt_two.at[615,'PCON11DCON']=0
join_attempt_two.at[615,'PCON11DMOH']=0

In [212]:
join_attempt_two[join_attempt_two['join_col']=='059_000700']

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge
615,059_000700,"POLYGON Z ((-77.31648 38.84615 0.00000, -77.31...",0,0,0,0,0,0,0,NaN,NaN,059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000700,Fairfax Court,left_only


## Look into split precincts

In [213]:
def is_split_precinct(district_assignment_list):
    c = Counter([x[0] for x in district_assignment_list])
    greater_than_one = {x:[y[1] for y in district_assignment_list if y[0]==x] for x, count in c.items() if count > 1}
    if len(greater_than_one)==0:
        return 0
    else:
        return greater_than_one
    
def get_level_dist(column_name):
    if column_name[0:4] == "GSLP":
        level = "SL"
        dist = "004"
    elif column_name[0:3] == "GSL":
        level = "SL"
        dist = column_name[3:6]
    elif column_name[0:3] == "PCO":
        level = "CON"
        dist = column_name[4:6]
    else:
        print(column_name)
        raise ValueError
    return level,dist

def contains_sldl(dist_list):
    for dist_tuple in dist_list:
        if dist_tuple[0] == "SL":
            return dist_tuple[1]
        
def contains_cong(dist_list):
    for dist_tuple in dist_list:
        if dist_tuple[0] == "CON":
            return dist_tuple[1]

precinct_mapping_dict = {}
split_precincts_list = {}
for index,row in join_attempt_two.iterrows():
    precinct_list = []
    for contest in con_races:
        if(row[contest]!=0):
            precinct_info = get_level_dist(contest)
            if precinct_info not in precinct_list:
                precinct_list.append(get_level_dist(contest))
    is_split = is_split_precinct(precinct_list)
    if (is_split):
        split_precincts_list[row["join_col"]]=is_split
    precinct_mapping_dict[row["join_col"]]=precinct_list

In [214]:
split_precincts_list

{'095_000202': {'CON': ['02', '01']},
 '153_000207': {'CON': ['01', '11']},
 '153_000210': {'CON': ['01', '11']},
 '153_000311': {'CON': ['01', '11']},
 '153_000312': {'CON': ['01', '11']},
 '153_000609': {'CON': ['01', '11']},
 '650_000215': {'CON': ['02', '03']},
 '700_000211': {'CON': ['02', '03']},
 '700_000218': {'CON': ['02', '03']},
 '800_000302': {'CON': ['03', '04']},
 '800_000404': {'CON': ['03', '04']},
 '800_000504': {'CON': ['03', '04']},
 '800_000602': {'CON': ['03', '04']}}

In [215]:
va_cong_districts = gp.read_file("./raw-from-source/va_2010_congress_2016-01-07_2021-12-31/Mod_16_shpfile.shp")
va_cong_districts = va_cong_districts.to_crs(vest_shp.crs)
va_cong_districts.rename(columns={'District_N':"HOUSE_DISTRICT"},inplace=True)
va_cong_districts["geometry"] = va_cong_districts["geometry"].buffer(0)

In [216]:
va_cong_districts["HOUSE_DISTRICT"] = va_cong_districts["HOUSE_DISTRICT"].astype(str)
join_attempt_two["join_col"] = join_attempt_two["join_col"].astype(str)
join_attempt_two["CONG_DIST"] = 0
join_attempt_two = gp.GeoDataFrame(join_attempt_two, crs = vest_shp.crs)

In [217]:
def return_cd(row):
    for val in con_races:
        if row[val] != 0:
            return val[4:6]

In [218]:
join_attempt_two["CONG_DIST"] = join_attempt_two.apply(lambda row: return_cd(row), axis = 1)

In [219]:
def district_splits_mod(cd_list, level, old_name, elections_gdf, shps_gdf, unique_ID_col, district_ID, races_list, dist_column):
    '''
    Function to split precincts across districts that splits a precinct across the entire district map.
    Previous iterations of this code only split precincts by the districts in which votes were recorded.
    In some instances, that led to holes in the map, due to districts where no votes were recorded in a precinct, but where an intersection occurred.
    '''
    full_shape = elections_gdf.loc[elections_gdf[unique_ID_col]==old_name]    
    test_join = gp.overlay(full_shape, shps_gdf, how = "intersection")
    test_join[dist_column] = test_join[district_ID]
    if test_join.empty:
        print("***Issue merging district ", old_name)
    for index, row in test_join.iterrows():
        
        test_join.at[index, unique_ID_col] = old_name+"-("+level + "-" + row[district_ID] + ")"
        for column in test_join:
            if column in races_list and row[district_ID] not in column:
                test_join.at[index, column] = 0 
        test_join = test_join[list(elections_gdf.columns)]
    elections_gdf=elections_gdf[elections_gdf[unique_ID_col]!=old_name]
    elections_gdf = pd.concat([elections_gdf, test_join])
    elections_gdf.reset_index(drop=True,inplace=True)
    return elections_gdf

In [220]:
for val in join_attempt_two["join_col"]:
    cd_list = []
    if val in split_precincts_list.keys():
        print(val)
        join_attempt_two = district_splits_mod(split_precincts_list[val],"CONG",val, join_attempt_two, va_cong_districts, "join_col", "HOUSE_DISTRICT", con_races, "CONG_DIST")

095_000202
153_000207
153_000210
153_000311
153_000312
153_000609
650_000215
700_000211
700_000218
800_000302
800_000404
800_000504
800_000602


In [221]:
join_attempt_two.shape

(2468, 57)

In [222]:
final_join.shape

(2454, 57)

In [223]:
join_attempt_two["CONG_DIST"] = join_attempt_two["CONG_DIST"].str.zfill(2)
join_attempt_two[join_attempt_two["CONG_DIST"].isna()]

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge,CONG_DIST
46,005_000101,"POLYGON Z ((-80.22172 37.62817 0.00000, -80.21...",154,720,0,3,231,636,0,101 - ARRITT,ALLEGHANY,005,0.0,3.0,0.0,23.0,1.0,1.0,2.0,32.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,0.0,5.0,0.0,54.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,ARRITT,both,None
47,005_000103,"POLYGON Z ((-80.29614 37.69236 0.00000, -80.29...",61,282,3,2,104,238,2,103 - DAMERON,ALLEGHANY,005,0.0,0.0,0.0,7.0,0.0,1.0,2.0,13.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,13.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000103,DAMERON,both,None
48,005_000201,"POLYGON Z ((-80.22950 37.79351 0.00000, -80.22...",102,380,4,3,139,343,0,201 - HUMPBACK BRIDGE,ALLEGHANY,005,0.0,1.0,0.0,16.0,0.0,0.0,7.0,23.0,0.0,0.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,28.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000201,HUMPBACK BRIDGE,both,None
49,005_000202,"POLYGON Z ((-80.00489 37.82628 0.00000, -80.00...",149,451,10,0,199,396,3,202 - DOLLY ANN,ALLEGHANY,005,0.0,3.0,0.0,19.0,0.0,0.0,10.0,41.0,0.0,0.0,0.0,5.0,3.0,0.0,2.0,0.0,0.0,4.0,0.0,41.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000202,DOLLY ANN,both,None
50,005_000301,"POLYGON Z ((-80.20117 37.82614 0.00000, -80.19...",63,462,4,0,108,414,2,301 - CALLAGHAN,ALLEGHANY,005,0.0,0.0,0.0,12.0,0.0,1.0,2.0,13.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,31.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000301,CALLAGHAN,both,None
51,005_000302,"POLYGON Z ((-80.06278 37.82277 0.00000, -80.06...",215,583,4,0,272,518,1,302 - GRIFFITH,ALLEGHANY,005,0.0,1.0,0.0,16.0,0.0,0.0,3.0,70.0,1.0,0.0,0.0,12.0,3.0,0.0,1.0,0.0,0.0,17.0,0.0,52.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000302,GRIFFITH,both,None
52,005_000401,"POLYGON Z ((-79.97473 37.75491 0.00000, -79.96...",117,401,6,0,151,359,0,401 - LOW MOOR,ALLEGHANY,005,0.0,3.0,0.0,10.0,0.0,1.0,14.0,49.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,20.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000401,LOW MOOR,both,None
53,005_000402,"POLYGON Z ((-80.03427 37.75014 0.00000, -80.03...",142,453,5,0,198,386,0,402 - JACKSON HEIGHTS,ALLEGHANY,005,0.0,4.0,0.0,16.0,0.0,0.0,7.0,50.0,0.0,0.0,1.0,5.0,0.0,1.0,2.0,0.0,0.0,12.0,0.0,44.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000402,JACKSON HEIGHTS,both,None
54,005_000501,"POLYGON Z ((-79.82905 37.79530 0.00000, -79.82...",140,218,4,2,162,192,0,501 - IRON GATE,ALLEGHANY,005,0.0,0.0,0.0,11.0,0.0,0.0,8.0,27.0,0.0,0.0,1.0,4.0,2.0,0.0,1.0,0.0,0.0,13.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000501,IRON GATE,both,None
55,005_000502,"POLYGON Z ((-79.88892 37.88022 0.00000, -79.88...",280,775,8,1,346,679,0,502 - PETERS SWITCH,ALLEGHANY,005,0.0,2.0,3.0,16.0,0.0,0.0,11.0,60.0,1.0,0.0,0.0,12.0,0.0,1.0,1.0,0.0,0.0,9.0,0.0,52.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000502,PETERS SWITCH,both,None


In [224]:
join_attempt_two.head(1)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge,CONG_DIST
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837,1618,29,2,915,1563,3,101 - CHINCOTEAGUE,ACCOMACK,001,0.0,2.0,1.0,85.0,0.0,0.0,35.0,209.0,0.0,1.0,0.0,39.0,3.0,0.0,1.0,0.0,0.0,36.0,88.0,144.0,28.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,CHINCOTEAGUE,both,02


## Setup Statewide file

In [225]:
final_join.head(1)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge,CONG_DIST
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837,1618,29,2,915,1563,3,101 - CHINCOTEAGUE,ACCOMACK,001,0.0,2.0,1.0,85.0,0.0,0.0,35.0,209.0,0.0,1.0,0.0,39.0,3.0,0.0,1.0,0.0,0.0,36.0,88.0,144.0,28.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,CHINCOTEAGUE,both,0


In [226]:
final_join.columns

Index(['join_col', 'geometry', 'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR',
       'G20PREOWRI', 'G20USSDWAR', 'G20USSRGAD', 'G20USSOWRI', 'PrecinctName',
       'LocalityName', 'COUNTYFP', 'P20PREOWRI', 'P20PREDKLO', 'P20PREDYAN',
       'P20PREDSAN', 'P20PREDBOO', 'P20PREDPAT', 'P20PREDWAR', 'P20PREDBID',
       'P20PREDCAS', 'P20PREDWIL', 'P20PREDBEN', 'P20PREDBLO', 'P20PREDBUT',
       'P20PREDSTE', 'P20PREDGAB', 'PCON02OWRI', 'P20USSOWRI', 'P20USSRBAL',
       'PCON02RLOY', 'P20USSRGAD', 'PCON02RBEL', 'PCON02RTAY', 'P20USSRSPE',
       'PCON03OWRI', 'PCON03RYAC', 'PCON03RDOW', 'PCON03RCOL', 'PCON05OWRI',
       'PCON05DWEB', 'PCON05DRUS', 'PCON05DLES', 'PCON05DHUF', 'PCON01OWRI',
       'PCON01DWIL', 'PCON01DRAS', 'PCON04OWRI', 'PCON04DMCE', 'PCON04DLEV',
       'PCON11OWRI', 'PCON11DCON', 'PCON11DMOH', 'VTDST', 'Precinct', '_merge',
       'CONG_DIST'],
      dtype='object')

In [227]:
final_join = final_join[['join_col', 'COUNTYFP','VTDST', 'Precinct',  
       'P20PREOWRI', 'P20PREDKLO', 'P20PREDYAN',
       'P20PREDSAN', 'P20PREDBOO', 'P20PREDPAT', 'P20PREDWAR', 'P20PREDBID',
       'P20PREDCAS', 'P20PREDWIL', 'P20PREDBEN', 'P20PREDBLO', 'P20PREDBUT',
       'P20PREDSTE', 'P20PREDGAB', 'P20USSOWRI', 'P20USSRBAL', 'P20USSRGAD','P20USSRSPE','geometry']]

In [228]:
final_join = final_join.rename(columns={"join_col": "UNIQUE_ID"})

In [229]:
final_join.sum(axis=0)

UNIQUE_ID     001_000101001_000201001_000202001_000301001_00...
COUNTYFP      0010010010010010010010010010010010010010010010...
VTDST         0001010002010002020003010004010004020004030005...
Precinct      CHINCOTEAGUEATLANTICGREENBACKVILLENEW CHURCHBL...
P20PREOWRI                                                184.0
P20PREDKLO                                               8414.0
P20PREDYAN                                               3361.0
P20PREDSAN                                             306388.0
P20PREDBOO                                               1910.0
P20PREDPAT                                                370.0
P20PREDWAR                                             142546.0
P20PREDBID                                             705501.0
P20PREDCAS                                                691.0
P20PREDWIL                                                902.0
P20PREDBEN                                               1437.0
P20PREDBLO                              

In [230]:
if not os.path.exists('./va_prim_20_prec/'):
    os.mkdir('./va_prim_20_prec/')

final_join.to_file('./va_prim_20_prec/va_prim_20_st_prec.shp')

## Setup Congressional File

In [231]:
join_attempt_two.head(1)

,join_col,geometry,G20PREDBID,G20PRERTRU,G20PRELJOR,G20PREOWRI,G20USSDWAR,G20USSRGAD,G20USSOWRI,PrecinctName,LocalityName,COUNTYFP,P20PREOWRI,P20PREDKLO,P20PREDYAN,P20PREDSAN,P20PREDBOO,P20PREDPAT,P20PREDWAR,P20PREDBID,P20PREDCAS,P20PREDWIL,P20PREDBEN,P20PREDBLO,P20PREDBUT,P20PREDSTE,P20PREDGAB,PCON02OWRI,P20USSOWRI,P20USSRBAL,PCON02RLOY,P20USSRGAD,PCON02RBEL,PCON02RTAY,P20USSRSPE,PCON03OWRI,PCON03RYAC,PCON03RDOW,PCON03RCOL,PCON05OWRI,PCON05DWEB,PCON05DRUS,PCON05DLES,PCON05DHUF,PCON01OWRI,PCON01DWIL,PCON01DRAS,PCON04OWRI,PCON04DMCE,PCON04DLEV,PCON11OWRI,PCON11DCON,PCON11DMOH,VTDST,Precinct,_merge,CONG_DIST
0,001_000101,"POLYGON Z ((-75.42507 37.89957 0.00000, -75.42...",837,1618,29,2,915,1563,3,101 - CHINCOTEAGUE,ACCOMACK,001,0.0,2.0,1.0,85.0,0.0,0.0,35.0,209.0,0.0,1.0,0.0,39.0,3.0,0.0,1.0,0.0,0.0,36.0,88.0,144.0,28.0,125.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000101,CHINCOTEAGUE,both,02


In [232]:
join_attempt_two.columns

Index(['join_col', 'geometry', 'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR',
       'G20PREOWRI', 'G20USSDWAR', 'G20USSRGAD', 'G20USSOWRI', 'PrecinctName',
       'LocalityName', 'COUNTYFP', 'P20PREOWRI', 'P20PREDKLO', 'P20PREDYAN',
       'P20PREDSAN', 'P20PREDBOO', 'P20PREDPAT', 'P20PREDWAR', 'P20PREDBID',
       'P20PREDCAS', 'P20PREDWIL', 'P20PREDBEN', 'P20PREDBLO', 'P20PREDBUT',
       'P20PREDSTE', 'P20PREDGAB', 'PCON02OWRI', 'P20USSOWRI', 'P20USSRBAL',
       'PCON02RLOY', 'P20USSRGAD', 'PCON02RBEL', 'PCON02RTAY', 'P20USSRSPE',
       'PCON03OWRI', 'PCON03RYAC', 'PCON03RDOW', 'PCON03RCOL', 'PCON05OWRI',
       'PCON05DWEB', 'PCON05DRUS', 'PCON05DLES', 'PCON05DHUF', 'PCON01OWRI',
       'PCON01DWIL', 'PCON01DRAS', 'PCON04OWRI', 'PCON04DMCE', 'PCON04DLEV',
       'PCON11OWRI', 'PCON11DCON', 'PCON11DMOH', 'VTDST', 'Precinct', '_merge',
       'CONG_DIST'],
      dtype='object')

In [233]:
join_attempt_two = join_attempt_two[['join_col', 'COUNTYFP','VTDST', 'Precinct','CONG_DIST',
         'PCON02OWRI', 'PCON02RLOY',  'PCON02RBEL', 'PCON02RTAY', 
       'PCON03OWRI', 'PCON03RYAC', 'PCON03RDOW', 'PCON03RCOL', 'PCON05OWRI',
       'PCON05DWEB', 'PCON05DRUS', 'PCON05DLES', 'PCON05DHUF', 'PCON01OWRI',
       'PCON01DWIL', 'PCON01DRAS', 'PCON04OWRI', 'PCON04DMCE', 'PCON04DLEV',
       'PCON11OWRI', 'PCON11DCON', 'PCON11DMOH', 'geometry']]

In [234]:
join_attempt_two = join_attempt_two.rename(columns={"join_col": "UNIQUE_ID"})

In [235]:
join_attempt_two.sum(axis=0)

UNIQUE_ID     001_000101001_000201001_000202001_000301001_00...
COUNTYFP      0010010010010010010010010010010010010010010010...
VTDST         0001010002010002020003010004010004020004030005...
Precinct      CHINCOTEAGUEATLANTICGREENBACKVILLENEW CHURCHBL...
PCON02OWRI                                                 24.0
PCON02RLOY                                              15420.0
PCON02RBEL                                              11616.0
PCON02RTAY                                              25478.0
PCON03OWRI                                                 20.0
PCON03RYAC                                               5853.0
PCON03RDOW                                               7816.0
PCON03RCOL                                               9004.0
PCON05OWRI                                                 10.0
PCON05DWEB                                              35965.0
PCON05DRUS                                               9833.0
PCON05DLES                              

In [236]:
if not os.path.exists('./va_prim_20_prec/'):
    os.mkdir('./va_prim_20_prec/')

join_attempt_two.to_file('./va_prim_20_prec/va_prim_20_cong_prec.shp')

## Setup README file

In [237]:
'''Following Docstring Convention: https://www.python.org/dev/peps/pep-0257/'''

import pandas as pd
import os
#THESE SETTINGS ESSENTIAL TO HAVE THE FIELDS TABLE SHOW UP CORRECTLY in the readme
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    '''Purpose: Create fields table used in readme based on field dictionary created separately
    Arguments:
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
    '''
    fields_table_data = {'Field Name': ['',race_field_header_0]  + list(fields_dict_0.keys()) +
                         ['',add_race_field_header_1] + list(fields_dict_1.keys()) +
                         ['',add_race_field_header_2] + list(fields_dict_2.keys()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.keys()),
                         'Description': ['',''] + list(fields_dict_0.values()) + 
                         ['',''] + list(fields_dict_1.values()) + 
                         ['',''] + list(fields_dict_2.values()) +
                         ['',''] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table

def erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, 
                        raw_data_source, state_erj_repo, office_codes, 
                        race_field_header_0, fields_dict_0, add_race_field_header_1 = '', fields_dict_1 = {}, add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {},
                       additional_notes=' '):
    '''Purpose: standardize language in ERJ file README.txt
    Arguemts:
        fields_dict = used to create the fields table for the non-standardized/race fields fields. 
                    Key is the field/value is the field description
        stateabrv = two character state abbreviation capitalized, 
        state = state name, first letter capitalized, 
        year = election year (XXXX), 
        election_type = general, primary, special or runoff, 
        additional_races = the races that RDH added to the original vest file (not in VEST's og file), 
        retrieval_date = date RDH retrieved VEST file, 
        vest_file_link = link to dataverse page for VEST's precinct boundary and election results file, 
        raw_data_source = site description and link, 
        state_erj_repo = link to erj github repository for given state
        office_codes = codes used broken off of field names for easy viewing. 
            For SU/SL/CON, include ##, so SU## for office code
        race_field_header_0: include asterisks "***text***" and label first set of fields
        fields_dict_0: the default dictionary for the primary file (statewide)
        add_race_field_header_1: include asterisks to draw attention to section - ex: "***additional_race_file_name_fields***"
        fields_dict_1: additional fields to go under add_race_field_header_1 header
        add_race_field_header_2 and _3: same use as add_race_field_header_1 - include as needed
        fields_dict_2 and _3: same use as fields_dict_1 - include as needed associated with corresponding add_race_field_header section
        additional_notes = default set to empty, but fill in with string where applicable.
    '''
#First section of README
    readme_p1 = '''{year} {stateabrv} {election_type} Precinct Boundary and Election Results
## RDH Date Retrieval
{retrieval_date}
## Sources
The RDH retrieved the VEST {year} {election_type} precinct boundary and election results shapefile from [VEST's Harvard Dataverse]({vest_file_link})
The RDH retrieved raw {year} {election_type} election results from {raw_data_source}
## Notes on Field Names (adapted from VEST):
Columns reporting votes generally follow the pattern: 
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.*
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.
One example is:
G16PREDCLI
To fit within the GIS 10 character limit for field names, the naming convention is slightly different for the State Legislature and 
US House of Representatives. All fields are listed below with definitions.
Office Codes Used:
{office_codes}
## Fields:
'''.format(stateabrv= stateabrv, state= state, year=year, election_type=election_type, additional_races=additional_races,retrieval_date=retrieval_date, vest_file_link=vest_file_link, raw_data_source=raw_data_source, state_erj_repo=state_erj_repo, office_codes=office_codes)

#Second section of README
    fields_table = create_fields_table(race_field_header_0, fields_dict_0, add_race_field_header_1, fields_dict_1, add_race_field_header_2, fields_dict_2, 
                        add_race_field_header_3, fields_dict_3)
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False)

#Third section of README
    readme_p3 = '''
## Processing Steps
    
The RDH joined additional election results to VEST's existing precinct shapefile, including {additional_races} using Python.
For more information on the processing completed, visit our [Github repository]({state_erj_repo}) for Election Result Joins (ERJ) for {state}.
Where possible, the RDH validated the election results we processed against VEST's election results. For additional races the RDH manually checked state totals. For more information on this comparison, please see our processing on Github ({state_erj_repo}).
## Additional Notes
{additional_notes}
Please contact info@redistrictingdatahub.org for more information.
'''.format(stateabrv=stateabrv, state=state, year=year, election_type=election_type, additional_races=additional_races, state_erj_repo=state_erj_repo, office_codes=office_codes, additional_notes = additional_notes)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

def export_readme(readme_name, state, election_type, full_readme_text):
    '''readme_name must include file path to readme within erj folder
    ex: 
    readme_name = './az_gen_20_prec/readme.txt'
    '''
    with open(readme_name, 'x') as tf:
        tf.write(full_readme_text)
    print(state, election_type, " readme moved to folder")

In [238]:
race_field_header_0 =  '***va_prim_20_st_prec.shp***'

fields_dict_0 = {'UNIQUE_ID':'Unique ID for the precinct, which takes into account splits if applicable', 
                 'COUNTYFP':'3-Digit county FIPS Code',
                 'VTDST':'Voting District Number', 
                 'Precinct':'Precinct Name',  
                 'P20PREOWRI':'Write-in Votes', 
                 'P20PREDKLO':'Amy J. Klobuchar (Democratic Party)', 
                 'P20PREDYAN':'Andrew Yang (Democratic Party)',
                 'P20PREDSAN':'Bernie Sanders (Democratic Party)', 
                 'P20PREDBOO':'Cory A. Booker (Democratic Party)', 
                 'P20PREDPAT':'Deval Patrick (Democratic Party)', 
                 'P20PREDWAR':'Elizabeth Warren (Democratic Party)', 
                 'P20PREDBID':'Joseph Robinette Biden Jr. (Democratic Party)',
                 'P20PREDCAS':'Juli Castro (Democratic Party)', 
                 'P20PREDWIL':'Marianne D. Williamson (Democratic Party)', 
                 'P20PREDBEN':'Michael F. Bennet (Democratic Party)', 
                 'P20PREDBLO':'Michael R. Bloomberg (Democratic Party)', 
                 'P20PREDBUT':'Peter M. P. Buttigieg (Democratic Party)',
                 'P20PREDSTE':'Thomas Steyer (Democratic Party)', 
                 'P20PREDGAB':'Tulsi Gabbard (Democratic Party)', 
                 'P20USSOWRI': 'Write-in Votes', 
                 'P20USSRBAL':'Alissa Baldwin (Republican Party)', 
                 'P20USSRGAD':'Daniel Gade (Republican Party)',
                 'P20USSRSPE':'Thomas Speciale (Republican Party)'}
                        
add_race_field_header_1 = '***va_prim_20_cong_prec.shp***' 

fields_dict_1 = {'CONG_DIST':'Congressional District Number',
                 'PCON02OWRI':'Write-in Votes', 
                 'PCON02RLOY':'Ben Loyola (Republican Party)',  
                 'PCON02RBEL':'Jarome Bell (Republican Party)', 
                 'PCON02RTAY':'Scott Taylor (Republican Party)', 
                 'PCON03OWRI':'Write-in Votes', 
                 'PCON03RYAC':'George Yacus (Republican Party)', 
                 'PCON03RDOW':'Madison Downs (Republican Party)', 
                 'PCON03RCOL':'John Collick (Republican Party)', 
                 'PCON05OWRI':'Write-in Votes',
                 'PCON05DWEB':'Cameron Webb (Democratic Party)', 
                 'PCON05DRUS':'Claire Russo (Democratic Party)', 
                 'PCON05DLES':'John Lesinski (Democratic Party)', 
                 'PCON05DHUF':'Roger Dean Huffstetler (Democratic Party)', 
                 'PCON01OWRI':'Write-in Votes',
                 'PCON01DWIL':'Vangie Williams (Democratic Party)', 
                 'PCON01DRAS':'Qasim Rashid (Democratic Party)', 
                 'PCON04OWRI':'Write-in Votes', 
                 'PCON04DMCE':'Aston Donald McEachin (Democratic Party)', 
                 'PCON04DLEV':'R. Cazel Levine (Democratic Party)',
                 'PCON11OWRI':'Write-in Votes', 
                 'PCON11DCON':'Gerald Edward Connolly (Democratic Party)', 
                 'PCON11DMOH':'Zainab Mohsini (Democratic Party)'
                }

add_race_field_header_2 = ''
fields_dict_2 = {}
add_race_field_header_3 = ''
fields_dict_3 = {}

In [239]:
stateabrv = "VA"
state = "Virginia"
year = "2020"
election_type = "primary"
additional_races = "U.S. House"
retrieval_date = "04/25/2022\n" 
vest_file_link = "https://dataverse.harvard.edu/file.xhtml?fileId=6174181&version=35.0"
raw_data_source = "Election results come from the Virginia Department of Elections(https://historical.elections.virginia.gov/) and precinct shapefiles from VEST's Virginia 2020 file.\n" 
state_erj_repo = "https://github.com/nonpartisan-redistricting-datahub/erj-va"

office_codes =  """PRE - President of the United States
USS - U.S. Senate
CON## - U.S. House
"""  


additional_notes = '''
Election Results matched State Wide totals except for the district 11 Congressional race.

'''

In [240]:
full_readme = erj_readme_template(stateabrv, state, year, election_type, additional_races, retrieval_date, vest_file_link, raw_data_source, state_erj_repo, office_codes, 
                        race_field_header_0, fields_dict_0, add_race_field_header_1, fields_dict_1, add_race_field_header_2, fields_dict_2, add_race_field_header_3, fields_dict_3,additional_notes)

In [241]:
export_readme("./va_prim_20_prec/README.txt", "Virginia", "primary", full_readme)

FileExistsError: [Errno 17] File exists: './va_prim_20_prec/README.txt'

In [ ]:
from zipfile import ZipFile

# more fine-grained control over ZIP files
with ZipFile("va_prim_20_prec.zip", "w") as newzip:
    for val in os.listdir("./va_prim_20_prec/"):
        newzip.write("./va_prim_20_prec/"+val)